In [ ]:
!uv pip install folium

## Import / Config

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import folium
from dotenv import dotenv_values

c_pg = dotenv_values("cred-pg.txt")


In [ ]:
def lat_lng_from_sql(sql, engine):
    with engine.connect() as con:
        lat_lng_df = pd.read_sql_query(text(sql), con)
    return lat_lng_df.iat[0,0], lat_lng_df.iat[0,1]

def geojson_from_sql(sql, engine):
    with engine.connect() as con:
        geojson_df = pd.read_sql_query(text(sql), con)
    return geojson_df.iat[0,0]
 

## Engine

In [ ]:
engine = create_engine(
    f'postgresql+psycopg://{c_pg["user"]}:{c_pg["password"]}@{c_pg["host"]}/{c_pg["dbname"]}', 
    connect_args = {
        'options': '-c search_path=${user},ugeobln,ugm,uinsta,umisc,umobility,usozmed,public', 
        'keepalives_idle': 120
    },
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [ ]:
# engine.dispose()

## HTW

In [ ]:
sql = """
select ST_Y(ST_Centroid(geometry)) as lat, ST_X(ST_Centroid(geometry)) as lng
from gis_osm_pois_a_free_1 
where osm_id ='41361350'; 
"""

lat, lng = lat_lng_from_sql(sql, engine)
print(f"{lat}, {lng}")
m = folium.Map(location=[lat, lng], zoom_start=16)
folium.Marker(
    location=[lat, lng],
    tooltip="HTW"
).add_to(m)

sql = """
select ST_AsGeoJSON(geometry)
from gis_osm_pois_a_free_1 
where osm_id ='41361350'; 
"""
geojson = geojson_from_sql(sql, engine)
folium.GeoJson(geojson).add_to(m)
m

## Bezirke

In [ ]:
m = folium.Map(location=[52.5170365, 13.3888599], zoom_start=10)

sql = """
select name as bname, ST_AsGeoJSON(shape) as geojson
from bezirk; 
"""
with engine.connect() as con:
    pdf = pd.read_sql_query(text(sql), con)

for b in pdf.apply(lambda r: folium.GeoJson(r.geojson), axis=1):
    b.add_to(m)

sql = """
select 
  name as bname, 
  ST_Y(ST_Centroid(shape)) as clat, 
  ST_X(ST_Centroid(shape)) as clng
from bezirk;
"""
with engine.connect() as con:
    pdf = pd.read_sql_query(text(sql), con)

def cmarker(r):
    loc = [r.clat, r.clng]
    html = f'<font color="green">{r.bname}</font>'
    divicon = folium.DivIcon(html=html)
    return folium.Marker(location=loc, icon=divicon)

for bc in pdf.apply(lambda r: cmarker(r), axis=1):
    bc.add_to(m)

sql = """
select ST_AsGeoJSON(ST_Extent(shape)) as bbox
from bezirk; 
"""
bb_berlin = geojson_from_sql(sql, engine)
folium.GeoJson(bb_berlin).add_to(m)
m

## Haltestellen

In [ ]:
m = folium.Map(location=[52.5170365, 13.3888599], zoom_start=13)
sql = """
select ST_AsGeoJSON(
    ST_Transform((st_dump(ST_VoronoiPolygons(
        st_collect(h.posp)))).geom, 4326)
    ) as geojson
from haltestelle h
"""
with engine.connect() as con:
    pdf = pd.read_sql_query(text(sql), con)

for v in pdf.apply(lambda r: folium.GeoJson(r.geojson), axis=1):
    v.add_to(m)

sql = """
select bez, lat, lng
from haltestelle;
"""
with engine.connect() as con:
    pdf = pd.read_sql_query(text(sql), con)

def hcircle(r):
    loc = [r.lat, r.lng]
    return folium.Circle(
        location=loc, radius=20, 
        fill=True, fill_color="blue", 
        tooltip=r.bez
    )

for h in pdf.apply(lambda r: hcircle(r), axis=1):
    h.add_to(m)
m